In [29]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Already up to date.


Load Dataset

In [38]:
from google.colab import drive
drive.mount('/content/drive')

url= "/content/drive/MyDrive/output/laptop14/test.tsv"
import pandas as pd

# Step 3: 读取TSV文件
file_path = url
df = pd.read_csv(file_path, sep='\t')
df_first_200 = df.head(200)
df_last = df.iloc[201:]
print(df_first_200.head())
print(df_last.head())
# Assuming the original DataFrame is named df
# Extracting the 'review' and 'aspect' columns
reviews = df_first_200['review'].tolist()
aspects = df_first_200['aspect'].tolist()
sentiments = df_first_200['sentiment'].tolist()

# Combining the extracted columns into the desired format
dev = [(review, aspect, "neutral" if sentiment == 0 else ("positive" if sentiment == 1 else "negative")) for review, aspect, sentiment in zip(reviews, aspects, sentiments)]

# Converting to the desired format using dspy.Example
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

# Print the result to verify
# for example in dev:
#     print(example)

############train dataset###########
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

#
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                              review        aspect  sentiment
0  Boot time is super fast, around anywhere from ...     Boot time          1
1  tech support would not fix the problem unless ...  tech support         -1
2                                   Set up was easy.        Set up          1
3  Did not enjoy the new Windows 8 and touchscree...     Windows 8         -1
4  Other than not being a fan of click pads (indu...    click pads          1
                                                review             aspect  \
201   OSX Mountain Lion  soon to upgrade to Mavericks.  OSX Mountain Lion   
202  I just bought the new MacBook Pro, the 13" mod...          USB ports   
203  It wakes in less than a second when I open the...              wakes   
204           It is the perfect size and speed for me.               size   
205               

Define a metric

In [32]:
def detect_predicted_label(pred):
    # 将预测结果转换为小写
    pred_sentiment = pred.sentiment.lower()

    # 初始化 predicted_label 为 "unknown"
    predicted_label = "unknown"

    # 检查预测结果中的特定字符串并分类，以第一个出现的为准
    if "ositiv" in pred_sentiment:
        predicted_label = "positive"
    elif "egativ" in pred_sentiment:
        predicted_label = "negative"
    elif "eutra" in pred_sentiment:
        predicted_label = "neutral"
    elif "ositiv" and "egativ" and "eutra" in pred_sentiment:
        predicted_label = "neutral"

    return predicted_label

In [16]:
def sentiment_classification_metric(example, pred, trace=None):
    predicted_label = detect_predicted_label(pred)

    # 检查预测结果是否与实际结果相同
    return example.sentiment.lower() == predicted_label

Design a signature using GPT3.5

In [6]:
import dspy
import openai
from dspy import OpenAI
import json

# openai.api_base = "https://api.gpts.vin/v1"
# openai.api_key = "sk-Pee7ZqyWixn655GY99Ff3eB6De9a4269Bf68E89505B90d48"

openai.api_base = "https://api.xiaoai.plus/v1"
openai.api_key = "sk-PZ3pu8tJkRrYv09o2aF29002270147809784D43e8241447d"


lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class AspectBasedSentimentAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")

Design a signature using GPT4.0

In [33]:
import dspy
import openai
from dspy import OpenAI
import json

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-Pee7ZqyWixn655GY99Ff3eB6De9a4269Bf68E89505B90d48"

# openai.api_base = "https://api.xiaoai.plus/v1"
# openai.api_key = "sk-PZ3pu8tJkRrYv09o2aF29002270147809784D43e8241447d"


lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-4o')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class AspectBasedSentimentAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")

Design a model

In [13]:
# 定义GPT3 DSPy模块
class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.Predict(AspectBasedSentimentAnalysis)
    def forward(self, sentence, aspect):
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect)
        print(f"Prediction:", prediction)
        return dspy.Prediction(sentiment=prediction.sentiment)
    def analyze_sentiment(self, sentence, aspect):
        prompt = f"Please select the following sentiment polarity of '{aspect}' in {sentence}. Positive, negative or neutral: "
        response = lm.complete(prompt=prompt)
        sentiment = response.choices[0].message['content'].strip().lower()
        return sentiment

In [34]:
# 定义GPT4 DSPy模块
class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.Predict(AspectBasedSentimentAnalysis)
    def forward(self, sentence, aspect):
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect)
        print(f"Prediction:", prediction)
        return dspy.Prediction(sentiment=prediction.sentiment)
    def analyze_sentiment(self, sentence, aspect):
        prompt = f"Please select the following sentiment polarity of '{aspect}' in {sentence}. Positive, negative, or neutral:"
        response = lm.complete(prompt=prompt)
        # 提取预测结果的情感标签
        sentiment = response.choices[0].message['content'].strip().lower()
        if "ositiv" in sentiment:
            return dspy.Prediction(sentiment='positive')
        elif "egativ" in sentiment:
            return dspy.Prediction(sentiment='negative')
        elif "eutra" in sentiment:
            return dspy.Prediction(sentiment='neutral')
        else:
            return dspy.Prediction(sentiment='unknown')

Evaluation

In [39]:
# 初始化情感分析器
analyzer = SentimentAnalyzer()

# 预测情感并计算准确率
correct_predictions = 0

for example in dev:
    try:
        prediction = analyzer(example['sentence'], example['aspect'])
        predicted_label = detect_predicted_label(prediction)
        actual_label = example.sentiment

        if predicted_label == actual_label:
            correct_predictions += 1
    except Exception as e:
        print(f"Error processing example: {example}")
        print(f"Exception: {str(e)}")
        continue

# 计算准确率
accuracy = correct_predictions / len(dev)

# 输出准确率并写入文件
dataset_name = "laptop14"  # 修改为实际的数据集名称
method_name = "DGPT4"  # 修改为实际的方法名称

output_file_path = f'/content/drive/MyDrive/DsPy/Results/laptop14/{dataset_name}_{method_name}_accuracy0.txt'
with open(output_file_path, 'w') as f:
    f.write(f"Correct Predictions: {correct_predictions}\n")
    f.write(f"Total Predictions: {len(dev)}\n")
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")

Prediction: Prediction(
    sentiment='---\nSentence: Boot time is super fast, around anywhere from 35 seconds to 1 minute.\nAspect: Boot time\nSentiment: positive\n---'
)
Prediction: Prediction(
    sentiment='---\n\nSentence: tech support would not fix the problem unless I bought your plan for $150 plus.\nAspect: tech support\nSentiment: negative\n\n---'
)
Prediction: Prediction(
    sentiment='---\nSentence: Set up was easy.\nAspect: Set up\nSentiment: positive'
)
Prediction: Prediction(
    sentiment='Sentence: Did not enjoy the new Windows 8 and touchscreen functions.\nAspect: Windows 8\nSentiment: negative'
)
Prediction: Prediction(
    sentiment="Sentence: Other than not being a fan of click pads (industry standard these days) and the lousy internal speakers, it's hard for me to find things about this notebook I don't like, especially considering the $350 price tag.\nAspect: click pads\nSentiment: negative"
)
Prediction: Prediction(
    sentiment='---\n\nSentence: No installatio